In [1]:
import serial
import time
import numpy as np

#pyqtgraph -> fast plotting
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui

#matplotlib -> normal plotting
import matplotlib.pyplot as plt
%matplotlib inline

import copy

In [2]:
# Change the configuration file name
configFileName = 'mmw_pplcount_demo_default.cfg'

global CLIport
global Dataport

CLIport = {}
Dataport = {}

CLIport = serial.Serial('COM4', 115200)
if not(CLIport.is_open):
    CLIport.open()
Dataport = serial.Serial('COM3', 921600)
if not(Dataport.is_open):
    Dataport.open()

In [3]:
# Read the configuration file and send it to the board
config = [line.rstrip('\r\n') for line in open(configFileName)]
for i in config:
    CLIport.write((i+'\n').encode())
    print(i)
    time.sleep(0.01)

#close control port
CLIport.close()

dfeDataOutputMode 1
channelCfg 15 3 0
adcCfg 2 1
adcbufCfg 0 1 1 1 
profileCfg 0 77 30 7 62 0 0 60 1 128 2500 0 0 30
chirpCfg 0 0 0 0 0 0 0 1
chirpCfg 1 1 0 0 0 0 0 2
frameCfg 0 1 128 0 50 1 0
lowPower 0 1
guiMonitor 1 1 0 0
cfarCfg 6 4 4 4 4 16 16 4 4 50 62 0
doaCfg 600 1875 30 1
SceneryParam -6 6 0.05 6
GatingParam 4 3 2 0
StateParam 10 5 10 100 5
AllocationParam 450 0.01 25 1 2
VariationParam 0.289 0.289 1.0
PointCloudEn 1
trackingCfg 1 2 250 20 200 50 90
sensorStart


In [4]:
#initialise variables
lostSync = False

#valid header variables and constant
magicBytes = np.array([2,1,4,3,6,5,8,7], dtype= 'uint8')

isMagicOk = False
isDataOk = False
gotHeader = False

frameHeaderLength = 52 #52 bytes long
tlvHeaderLengthInBytes = 8
pointLengthInBytes = 16
frameNumber = 1
targetFrameNumber = 0



In [5]:
def validateChecksum(recieveHeader):
    uint32Convertor = [1, 2**8, 2**16, 2**24]
    uint16Convertor = uint32Convertor[0:2]
    h = np.uint16(recieveHeader)
    a = np.uint32(sum(h))
    b = np.uint16(a)
    CS = np.uint16(~(b))
    return CS

In [6]:
def readHeader(recieveHeader):
    headerContent = dict()
    index = 0
    
    headerContent['magicBytes'] = recieveHeader[index:index+8]
    index += 20
    
    headerContent['packetLength'] = recieveHeader[index:index+4].view(dtype=np.uint32)
    index += 4
        
    headerContent['frameNumber'] = recieveHeader[index:index+4].view(dtype=np.uint32)
    index += 24
    
    headerContent['numTLVs'] = recieveHeader[index:index+2].view(dtype=np.uint16)
    
    return headerContent

In [7]:
def tlvParsing(data, dataLength, tlvHeaderLengthInBytes, pointLengthInBytes):
    
    
    index = 0
    #tlv header parsing
    tlvType = data[index:index+4].view(dtype=np.uint32)
    tlvLength = data[index+4:index+8].view(dtype=np.uint32)
    
    #TLV size check
    if (tlvLength + index > dataLength):
        print('TLV SIZE IS WRONG')
        lostSync = True
        return
    
    index += tlvHeaderLengthInBytes
    tlvDataLength = tlvLength - tlvHeaderLengthInBytes
    if tlvType == 6: #point cloud TLV
        numberOfPoints = tlvDataLength/pointLengthInBytes
        print('NUMBER OF POINTS ', str(int(numberOfPoints)))
        if numberOfPoints > 0:
            p = data[index:index+tlvDataLength[0]].view(dtype=np.single)
            #form the appropriate array 
            #each point is 16 bytes - 4 bytes for each property - range, azimuth, doppler, snr
            pointCloud = np.reshape(p,(4, int(numberOfPoints)))
            return pointCloud
        
    

In [ ]:
#read and parse data

while Dataport.is_open:
    
    while (not(lostSync) and Dataport.is_open):
        #check for a valid frame header
        if not(gotHeader):
            
            #in_waiting = amount of bytes in the buffer
            rawRecieveHeader = Dataport.read(frameHeaderLength)
            recieveHeader = np.frombuffer(rawRecieveHeader, dtype = 'uint8')
            
        #magic byte check
        if not(np.array_equal(recieveHeader[0:8],magicBytes)):
            print('NO SYNC PATTERN')
            lostSync = True
            break

        #valid the checksum
#         CS = validateChecksum(recieveHeader)
        
        #have a valid frame header
        headerContent = readHeader(recieveHeader)
        print(headerContent)
        
        if (gotHeader):
            if headerContent['frameNumber'] > targetFrameNumber:
                targetFrameNumber = headerContent['frameNumber']
                gotHeader = False
                print('FOUND SYNC AT FRAME NUMBER ' + str(targetFrameNumber))
            else:
                print('OLD FRAME')
                gotHeader = False
                lostSync = True
                break
                
        dataLength = int(headerContent['packetLength'] - frameHeaderLength)
        if dataLength > 0:
            #read the rest of the packet
            rawData = Dataport.read(dataLength)
            data = np.frombuffer(rawData, dtype = 'uint8')
            pointCloud = tlvParsing(data, dataLength, tlvHeaderLengthInBytes, pointLengthInBytes)
            
            plt.tile
            
            
            
            

{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([52], dtype=uint32), 'frameNumber': array([21842], dtype=uint32), 'numTLVs': array([0], dtype=uint16)}
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([52], dtype=uint32), 'frameNumber': array([21843], dtype=uint32), 'numTLVs': array([0], dtype=uint16)}
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([52], dtype=uint32), 'frameNumber': array([21844], dtype=uint32), 'numTLVs': array([0], dtype=uint16)}
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([52], dtype=uint32), 'frameNumber': array([21845], dtype=uint32), 'numTLVs': array([0], dtype=uint16)}
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([52], dtype=uint32), 'frameNumber': array([21846], dtype=uint32), 'numTLVs': array([0], dtype=uint16)}
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLe

{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([52], dtype=uint32), 'frameNumber': array([21888], dtype=uint32), 'numTLVs': array([0], dtype=uint16)}
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([52], dtype=uint32), 'frameNumber': array([21889], dtype=uint32), 'numTLVs': array([0], dtype=uint16)}
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([52], dtype=uint32), 'frameNumber': array([21890], dtype=uint32), 'numTLVs': array([0], dtype=uint16)}
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([92], dtype=uint32), 'frameNumber': array([21891], dtype=uint32), 'numTLVs': array([1], dtype=uint16)}
NUMBER OF POINTS  2
[[ 1.3680205   0.45814893]
 [-0.08109608 12.91205   ]
 [ 1.3680205   0.425424  ]
 [ 0.08109608 15.793256  ]]
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([52], dtype=uint32), 'frameNumber': array([2

{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([92], dtype=uint32), 'frameNumber': array([21927], dtype=uint32), 'numTLVs': array([1], dtype=uint16)}
NUMBER OF POINTS  2
[[ 1.3680205   0.45814893]
 [ 0.08109608 14.229745  ]
 [ 1.3680205   0.425424  ]
 [ 0.08109608 13.874541  ]]
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([52], dtype=uint32), 'frameNumber': array([21928], dtype=uint32), 'numTLVs': array([0], dtype=uint16)}
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([52], dtype=uint32), 'frameNumber': array([21929], dtype=uint32), 'numTLVs': array([0], dtype=uint16)}
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([52], dtype=uint32), 'frameNumber': array([21930], dtype=uint32), 'numTLVs': array([0], dtype=uint16)}
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([52], dtype=uint32), 'frameNumber': array([2

{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([52], dtype=uint32), 'frameNumber': array([21964], dtype=uint32), 'numTLVs': array([0], dtype=uint16)}
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([92], dtype=uint32), 'frameNumber': array([21965], dtype=uint32), 'numTLVs': array([1], dtype=uint16)}
NUMBER OF POINTS  2
[[ 0.29314727  0.7853982 ]
 [-0.08109608  6.441597  ]
 [ 0.29314727  0.7526732 ]
 [-0.08109608  6.3566213 ]]
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([92], dtype=uint32), 'frameNumber': array([21966], dtype=uint32), 'numTLVs': array([1], dtype=uint16)}
NUMBER OF POINTS  2
[[0.29314727 0.7853982 ]
 [0.08109608 6.450336  ]
 [0.29314727 0.7526732 ]
 [0.08109608 6.000331  ]]
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([108], dtype=uint32), 'frameNumber': array([21967], dtype=uint32), 'numTLVs': array([1], dtype=uint16)}
NUMBE

NUMBER OF POINTS  3
[[ 0.24428937  0.7853982  -0.08109608]
 [ 5.0680804   0.29314727  0.7853982 ]
 [ 0.16219215  6.8870006   0.29314727]
 [ 0.7526732   4.054804    6.3398614 ]]
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([76], dtype=uint32), 'frameNumber': array([21992], dtype=uint32), 'numTLVs': array([1], dtype=uint16)}
NUMBER OF POINTS  1
[[1.4168783 ]
 [0.2945243 ]
 [0.08109608]
 [8.67554   ]]
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([52], dtype=uint32), 'frameNumber': array([21993], dtype=uint32), 'numTLVs': array([0], dtype=uint16)}
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([52], dtype=uint32), 'frameNumber': array([21994], dtype=uint32), 'numTLVs': array([0], dtype=uint16)}
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([52], dtype=uint32), 'frameNumber': array([21995], dtype=uint32), 'numTLVs': array([0], dtype=uint16)}
{'

...

None
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([128], dtype=uint32), 'frameNumber': array([23185], dtype=uint32), 'numTLVs': array([1], dtype=uint16)}
None
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([128], dtype=uint32), 'frameNumber': array([23186], dtype=uint32), 'numTLVs': array([1], dtype=uint16)}
None
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([128], dtype=uint32), 'frameNumber': array([23187], dtype=uint32), 'numTLVs': array([1], dtype=uint16)}
None
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([152], dtype=uint32), 'frameNumber': array([23188], dtype=uint32), 'numTLVs': array([2], dtype=uint16)}
NUMBER OF POINTS  1
[[ 1.6123099 ]
 [ 0.06544985]
 [-0.08109608]
 [ 7.663565  ]]
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([137], dtype=uint32), 'frameNumber': array([23189], dtype=uint32), '

{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([128], dtype=uint32), 'frameNumber': array([23228], dtype=uint32), 'numTLVs': array([1], dtype=uint16)}
None
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([184], dtype=uint32), 'frameNumber': array([23229], dtype=uint32), 'numTLVs': array([2], dtype=uint16)}
NUMBER OF POINTS  3
[[ 1.2703048  -0.35997415 -0.08109608]
 [14.073088    1.4657363   0.32724923]
 [-0.08109608 10.26057     1.2703048 ]
 [-0.32724923 -0.08109608 11.386156  ]]
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([227], dtype=uint32), 'frameNumber': array([23230], dtype=uint32), 'numTLVs': array([3], dtype=uint16)}
NUMBER OF POINTS  5
[[ 1.4168783   0.35997415 -0.08109608 15.401295    1.4657363 ]
 [ 0.35997415 -0.08109608 19.728077    1.4657363   0.32724923]
 [-0.08109608 22.628319    1.4657363   0.2945243  -0.08109608]
 [18.27084     1.4657363   0.2617994  -0.0810960

None
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([216], dtype=uint32), 'frameNumber': array([23250], dtype=uint32), 'numTLVs': array([2], dtype=uint16)}
NUMBER OF POINTS  5
[[ 1.4657363   0.45814893  0.08109608 15.523653    1.4657363 ]
 [ 0.425424    0.08109608 17.382263    1.5145941   0.425424  ]
 [ 0.08109608 25.146013    1.5145941   0.3926991   0.08109608]
 [31.524704    1.5145941   0.35997415 -0.08109608 20.099092  ]]
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([293], dtype=uint32), 'frameNumber': array([23251], dtype=uint32), 'numTLVs': array([3], dtype=uint16)}
NUMBER OF POINTS  9
[[ 1.4657363   0.45814893 -0.08109608 31.543709    1.4657363   0.425424
  -0.08109608 40.420395    1.5145941 ]
 [ 0.425424    0.08109608 42.17636     1.4657363   0.3926991  -0.08109608
  23.43539     1.5145941   0.3926991 ]
 [ 0.08109608 51.422615    1.5145941   0.35997415 -0.08109608 37.70394
   1.3191626   0.22907446  0.0810

{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([168], dtype=uint32), 'frameNumber': array([23262], dtype=uint32), 'numTLVs': array([2], dtype=uint16)}
NUMBER OF POINTS  2
[[ 1.3680205   0.2945243 ]
 [-0.08109608 11.151882  ]
 [ 1.3680205   0.2617994 ]
 [-0.08109608 13.191523  ]]
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([178], dtype=uint32), 'frameNumber': array([23263], dtype=uint32), 'numTLVs': array([3], dtype=uint16)}
NUMBER OF POINTS  2
[[ 1.3680205   0.2945243 ]
 [-0.08109608 10.426439  ]
 [ 1.3680205   0.2617994 ]
 [-0.08109608 10.64844   ]]
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([162], dtype=uint32), 'frameNumber': array([23264], dtype=uint32), 'numTLVs': array([3], dtype=uint16)}
NUMBER OF POINTS  1
[[ 1.4657363 ]
 [ 0.35997415]
 [ 0.08109608]
 [11.450841  ]]
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([137], dtype=uint

{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([180], dtype=uint32), 'frameNumber': array([23296], dtype=uint32), 'numTLVs': array([3], dtype=uint16)}
NUMBER OF POINTS  2
[[ 1.563452    0.425424  ]
 [-0.08109608 10.887495  ]
 [ 1.563452    0.3926991 ]
 [-0.08109608 10.950056  ]]
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([162], dtype=uint32), 'frameNumber': array([23297], dtype=uint32), 'numTLVs': array([3], dtype=uint16)}
NUMBER OF POINTS  1
[[ 1.4657363 ]
 [ 0.35997415]
 [-0.08109608]
 [12.6243105 ]]
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([137], dtype=uint32), 'frameNumber': array([23298], dtype=uint32), 'numTLVs': array([2], dtype=uint16)}
None
{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([128], dtype=uint32), 'frameNumber': array([23299], dtype=uint32), 'numTLVs': array([1], dtype=uint16)}
None
{'magicBytes': array([2, 1, 4, 3

In [40]:
print(pointCloud)

None


In [32]:
A = np.array([1,2,3])
B = np.reshape(A,(3,1))
            

In [98]:
x = np.array([89,153,34,63], dtype=np.uint8)
print(x.view(dtype=np.single))

[0.6351524]
